In [5]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

# ------------------------
# 1.  helper lag kernels
# ------------------------
def linear_kernel(t, t_ramp):
    """Linear rise that saturates after t_ramp."""
    return np.clip(t / t_ramp, 0.0, 1.0)

def quadratic_kernel(t, t_ramp):
    """Quadratic rise, saturating after t_ramp."""
    g = np.clip(t / t_ramp, 0.0, 1.0)
    return g**2

def exp_kernel(t, tau):
    """First-order step response: 1-exp(-t/τ)."""
    return 1.0 - np.exp(-t / tau)

# mapping so strings can be used in the API
KERNELS = {
    "linear":    (linear_kernel,  ("t_ramp", 30.0)),   # default 30 s
    "quadratic": (quadratic_kernel, ("t_ramp", 30.0)),
    "exp":       (exp_kernel, ("tau", 20.0)),          # default 20 s
}

# -------------------------------------------------
# 2.  main simulator
# -------------------------------------------------
def simulate_hashrate(
        t_base, p_base, h_base,
        p_new,
        kernel_name="exp",
        kernel_kwargs=None,
        t_max=120.0,
        dt=1.0,
        plot=False):
    """
    Parameters
    ----------
    t_base, p_base, h_base : 1-D arrays
        Baseline time, power, hashrate histories (same length).
    p_new : float
        Instantaneous step target (same units as p_base).
    kernel_name : str
        'exp', 'linear', or 'quadratic', or supply your own callable in kernel_kwargs.
    kernel_kwargs : dict or None
        Extra parameters for the chosen kernel.  If None, defaults are used.
    t_max : float
        Length of simulation window (seconds).
    dt : float
        Sampling period of the simulated curve.
    plot : bool
        If True, show a Matplotlib figure.

    Returns
    -------
    t_sim : 1-D array of shape (N,)
    h_sim : 1-D array of shape (N,)
    """
    # --- 2.1  estimate proportionality coefficient k from baseline
    #         use linear regression on the flat portion
    coeffs = np.polyfit(p_base, h_base, 1)
    k = coeffs[0]          # slope ΔH/ΔP
    h0 = h_base[-1]        # last measured hashrate
    p0 = p_base[-1]        # last measured power

    # --- 2.2  prepare kernel
    if callable(kernel_name):
        kernel_fn = kernel_name
        params    = kernel_kwargs or {}
    else:
        kernel_fn, (param_name, default) = KERNELS[kernel_name]
        params = {param_name: kernel_kwargs.get(param_name, default)} if kernel_kwargs else {param_name: default}

    # --- 2.3  simulate
    t_sim = np.arange(0.0, t_max + dt, dt)
    g     = kernel_fn(t_sim, **params)          # 0→1 growth curve
    h_step = k * (p_new - p0) * g               # dynamic increment
    h_sim  = h0 + h_step

    # --- 2.4  optional plot
    if plot:
        plt.figure()
        plt.plot(t_sim, h_sim, label="simulated hashrate")
        plt.axhline(h0, linestyle="--", linewidth=0.8)
        plt.axhline(h0 + k*(p_new-p0), linestyle="--", linewidth=0.8)
        plt.xlabel("time [s]")
        plt.ylabel("hashrate [same units as input]")
        plt.title(f"Step response ({kernel_name})")
        plt.legend()
        plt.grid(True)
        plt.show()

    return t_sim, h_sim


In [11]:
# example baseline
t_base = np.array([0, 10, 20, 30, 40, 50])        # seconds
p_base = np.full_like(t_base, 200.0)              # watts
h_base = np.full_like(t_base, 15.0)               # MH/s

# simulate +50 W step with an exponential lag (τ = 25 s)
t, h = simulate_hashrate(t_base, p_base, h_base,
                         p_new=250.0,
                         kernel_name="exp",
                         kernel_kwargs={"tau": 25.0},
                         t_max=120,
                         dt=1.0,
                         plot=True)


/var/folders/gc/x3bfhyjd6w5_brdjmcp7y05w0000gn/T/ipykernel_18843/1036657450.py:64: RankWarning: Polyfit may be poorly conditioned
  coeffs = np.polyfit(p_base, h_base, 1)


KeyError: 'quad'